In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pulses import RadarPulseGenerator, CommPulseGenerator
import torch
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)
t = torch.linspace(0, 1e-4, int(1e5), device=device)
dt = t[1] - t[0]
# Define parameters
T = 1e-4       # Total duration of radar pulse (1e-4 seconds)
B = 1e6        # Bandwidth of radar pulse (1 MHz)
K = 5         # Number of users (OFDMA users)
L = 100        # Number of bits in radar pulse duration
Tb = T / L     # Duration of each bit interval
carrier_freq = 800e6
SNR_db = 30

myCommPulses = CommPulseGenerator(t, Tb, T, device)
myRadar = RadarPulseGenerator(t, T=T, device=device)

radar_pulse = myRadar.LFM_pulse(B) * np.sqrt(1e4)
radar_pulse_power = myRadar.power_calc(radar_pulse)
print(10*np.log10(radar_pulse_power.cpu().numpy()))

N0 = 10**(-114/10)
print(10*np.log10(N0))
MN = int(1e4)
distances = np.linspace(1,1000,10)
est_distances = torch.zeros((MN, len(distances)))
for idx, dis in enumerate(distances):
    # print((idx+1)/len(distances)*100)
    A = myRadar.LOS_pathloss(dis, carrier_freq, 3e8)
    # print(f'distance: {dis}m, Loss: {A}')
    est_distances[:, idx], _ = myRadar.montecarlo_estimation_with_real(radar_pulse, N0, dis, np.sqrt(A), 3e8, MN)
    print(f'Mean value->{torch.mean(est_distances[:, idx])}, \t Variance->{torch.var(est_distances[:, idx])}')
# plt.plot(distances, torch.var(est_distances, dim=0).numpy())
# dis, ta = myRadar.montecarlo_estimation_with_abs(radar_pulse, N0, 204, np.sqrt(A), 3e8, 10000)
# print(f'Mean value->{torch.mean(dis)}, \t Variance->{torch.var(dis)}')
# print(f'Mean value->{torch.mean(ta)}, \t Variance->{torch.var(ta)}')

# dis, ta = myRadar.montecarlo_estimation_with_real(radar_pulse, N0, 204, np.sqrt(A), 3e8, 10000)
# print(f'Mean value->{torch.mean(dis)}, \t Variance->{torch.var(dis)}')
# print(f'Mean value->{torch.mean(ta)}, \t Variance->{torch.var(ta)}')

In [ ]:
error = (est_distances.cpu().numpy() - distances)**2
plt.plot(distances, np.mean(error, axis=0))
# plt.plot(distances, torch.var(est_distances, dim=0).cpu().numpy())

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Setup Parameters ---
fs = 50e6  # Sampling frequency (e.g., 10 MHz), must be > 2*B
dt = 1/fs
B = 1e6    # Bandwidth = 1 MHz
T = 1e-4   # Pulse duration = 1 ms
N = int(fs * T) # Number of samples in the pulse

# Time and frequency vectors
t = torch.arange(N, device='cuda') / fs
f_axis = torch.fft.fftfreq(N, 1/fs)

# --- 2. Generate Transmitted and Received Signals ---
# Transmitted LFM pulse s(t)
s_t = torch.exp(1j * torch.pi * B / T * (t)**2)
print(f'signal Power: {torch.sum(torch.abs(s_t)**2)*dt}')

# --- Create a simulated received signal r(t) with known delay and Doppler ---
true_delay = 2.5e-6  # 200 us delay
true_doppler = -1000 # 5 kHz Doppler shift
true_delay_samples = int(true_delay * fs)


# Create the received signal by shifting and modulating the transmitted one
# Note: torch.roll provides a simple circular shift for the delay
r_t_clean = torch.roll(s_t * torch.exp(1j * 2 * np.pi * true_doppler * t), shifts=true_delay_samples) 
# freqs = torch.fft.fftfreq(N, d=dt, device='cuda')
# r_t_clean = torch.fft.ifft(torch.fft.fft(s_t) * torch.exp(-1j * 2 * np.pi * freqs * true_delay)) * torch.exp(1j * 2 * np.pi * true_doppler * t)

# Add some noise
noise_power = 1
noise = torch.randn(N, dtype=torch.cfloat, device='cuda') * np.sqrt(noise_power / 2)
r_t = r_t_clean + noise

# --- 3. Fast Joint Estimation (The Loop-Free Part) ---
# Define the grid of Doppler shifts to search over
doppler_shifts = torch.linspace(-1000, 1000, 4001, device='cuda') # Search from -10 kHz to 10 kHz
print(doppler_shifts[1]-doppler_shifts[0])
doppler_exp = torch.exp(1j * 2 * np.pi * doppler_shifts[:, torch.newaxis] * t[torch.newaxis, :])
s_t_dopplers = s_t[torch.newaxis, :] * doppler_exp
# Step 1: FFT of signals
R = torch.fft.fft(r_t)
S = torch.fft.fft(s_t_dopplers, dim=1)
corr = torch.fft.ifft(R[torch.newaxis, :] * torch.conj(S), dim=1)
XCORR = torch.real(torch.fft.fftshift(corr, dim=1))*dt
arg_max_est = (XCORR==torch.max(XCORR)).nonzero()[0]
print(doppler_shifts[arg_max_est[0]])
print(t[arg_max_est[1]]-t[-1]/2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pulses import RadarPulseGenerator, CommPulseGenerator
import torch
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)
t = torch.linspace(0, 1e-4, int(1e4), device=device)
dt = t[1] - t[0]
print(dt)
# Define parameters
T = 1e-4       # Total duration of radar pulse (1e-4 seconds)
B = 1e6        # Bandwidth of radar pulse (1 MHz)
carrier_freq = 800e6

myRadar = RadarPulseGenerator(t, T=T, device=device)

radar_pulse = myRadar.LFM_pulse(B) * np.sqrt(1e4) #power = 1Watt

N0 = 10**(-200/10) #Nois Power = -114dB
MN = int(1e4) #MonteCarlo Number
distance = 100
A = myRadar.LOS_pathloss(distance, carrier_freq, 3e8)
est_delay, est_doppler = myRadar.montecarlo_estimation_doppler_delay(radar_pulse, N0, np.sqrt(A), 10000, 2*distance/3e8, 6)
print(f'Mean value->{torch.mean(est_delay)}, \t Variance->{torch.var(est_delay)}')
print(f'Mean value->{torch.mean(est_doppler)}, \t Variance->{torch.var(est_doppler)}')

  6%|▋         | 644/10000 [04:00<1:02:43,  2.49it/s]